# Ćwiczenie: Perceptron binarny
W tym ćwiczeniu zbudujesz i przetestujesz prosty perceptron do klasyfikacji otyłości na podstawie wzrostu i wagi. Postępuj krok po kroku zgodnie z instrukcjami w notatniku. Na końcu obliczysz dokładność na zestawie testowym.


In [ ]:
import numpy as np

# Funkcja aktywacji (progowa)
def threshold_function(x: np.array, threshold: int = 0) -> np.array:
    """
    Funkcja progowa (threshold function).

    Dla wartości wejściowej x:
    - zwraca 1, jeśli x >= threshold,
    - zwraca 0, jeśli x < threshold.

    Parametry
    ---------
    x : np.array lub float
        Wartość (lub tablica wartości) do sprawdzenia.
    threshold : int, domyślnie 0
        Próg aktywacji.

    Zwraca
    -------
    np.array
        Wartości 0 lub 1 po zastosowaniu funkcji progowej.
    """
    return np.where(x >= threshold, 1, 0)


class Perceptron:
    """
    Implementacja prostego perceptronu binarnego (uczenie nadzorowane).

    Perceptron uczy się liniowej granicy decyzyjnej dla klasyfikacji
    na podstawie wektorów cech i etykiet binarnych (0 lub 1).
    """

    def __init__(self, learning_rate=0.01, epochs=100,
                 activation_function=threshold_function,
                 start_weights=None, start_bias=None):
        """
        Inicjalizacja perceptronu.

        Parametry
        ---------
        learning_rate : float
            Współczynnik uczenia (krok aktualizacji wag).
        epochs : int
            Liczba epok, tj. ile razy przechodzimy przez cały zbiór treningowy.
        activation_function : callable
            Funkcja aktywacji (np. progowa, sigmoid).
        start_weights : np.array lub None
            Wagi początkowe (jeśli None, będą zainicjalizowane zerami).
        start_bias : float lub None
            Bias początkowy (jeśli None, startuje od 0).
        """
        # hiperparametry
        self.lr = learning_rate
        self.epochs = epochs
        self.activation_func = activation_function

        # parametry (uczone podczas treningu)
        self.weights = start_weights
        self.bias = start_bias or 0

    def fit(self, features, labels):
        """
        Trenuje perceptron na podanych danych.

        Parametry
        ---------
        features : np.array, shape (n_samples, n_features)
            Macierz cech (wejścia).
        labels : np.array lub lista, shape (n_samples,)
            Etykiety klas (0 lub 1).
        """
        n_samples, n_features = features.shape

        # Inicjalizacja wag zerami (jeśli nie podano startowych)
        self.weights = np.zeros(n_features)

        y = np.array(labels)

        # Pętla ucząca (epoki)
        for _ in range(self.epochs):
            # Iterujemy po wszystkich próbkach
            for i, x_i in enumerate(features):
                # Obliczamy wyjście liniowe
                linear_output = self._linear_output(x_i)

                # Przepuszczamy przez funkcję aktywacji
                y_predicted = self.activation_func(linear_output)

                # Obliczamy błąd (y_true - y_pred)
                # i wyznaczamy wielkość aktualizacji
                update = self.lr * (y[i] - y_predicted)

                # Aktualizacja wag i biasu
                self.weights += update * x_i
                self.bias += update

    def _linear_output(self, features):
        """
        Oblicza wyjście liniowe (iloczyn skalarny wejścia i wag + bias).

        Parametry
        ---------
        features : np.array, shape (n_features,)
            Wektor cech dla jednej próbki.

        Zwraca
        -------
        float
            Wartość liniowej kombinacji cech.
        """
        return features @ self.weights + self.bias
    
    def predict(self, features):
        """
        Przewiduje etykiety dla nowych danych.

        Parametry
        ---------
        features : np.array, shape (n_samples, n_features) lub (n_features,)
            Dane wejściowe.

        Zwraca
        -------
        np.array
            Predykcje (0 lub 1) dla próbek wejściowych.
        """
        linear_output = self._linear_output(features)
        y_predicted = self.activation_func(linear_output)
        return y_predicted


## Zadanie: wczytaj dane i przygotuj etykiety
Masz plik wzrost_waga.txt w tym samym katalogu. Każdy wiersz zawiera trzy wartości rozdzielone spacją lub tabem:
- wzrost w metrach (float),
- waga w kilogramach (float),
- etykieta klasy 0/1 (int), wyznaczona na podstawie BMI (próg 25).

Twoim zadaniem jest wczytać dane, wyodrębnić cechy X (wzrost, waga) i etykiety y (0/1).


In [29]:
# Funkcja pomocnicza do wczytania danych z pliku

def load_data(filepath: str):
    """
    Wczytuje dane z pliku tekstowego i zwraca:
    - X: np.ndarray o kształcie (n_samples, 2) [wzrost, waga]
    - y: np.ndarray o kształcie (n_samples,)
    Plik może być rozdzielany spacją lub tabem.
    """
    data = np.loadtxt(filepath)
    X = data[:, :2]
    y = data[:, 2].astype(int)
    return X, y


array([0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0])

In [23]:
# 1. Wczytaj dane
X, y = load_data("wzrost_waga.txt")
y

In [33]:
# 2. Utwórz i wytrenuj perceptron
perceptron = Perceptron(learning_rate=0.1, epochs=50)
perceptron.fit(X, y)
print("Wyuczone wagi:", perceptron.weights)
print("Wyuczony bias:", perceptron.bias)


[19.53125    22.20408163 27.77777778 29.38475666 26.31578947 20.76124567
 29.13631634]
X_test:
 [[ 1.6  50.  ]
 [ 1.75 68.  ]
 [ 1.8  90.  ]
 [ 1.65 80.  ]
 [ 1.9  95.  ]
 [ 1.7  60.  ]
 [ 1.55 70.  ]]
y_test: [0. 0. 1. 1. 1. 0. 1.]


In [ ]:
# Sprawdź na danych testowych, jak działa model
# Dane: [wzrost (m), waga (kg), klasa]
# Klasa obliczona na podstawie BMI (próg = 25)
data = np.array([
    [1.60, 50, 0],   # BMI = 19.5 → klasa 0
    [1.75, 68, 0],   # BMI = 22.2 → klasa 0
    [1.80, 90, 1],   # BMI = 27.8 → klasa 1
    [1.65, 80, 1],   # BMI = 29.4 → klasa 1
    [1.90, 95, 1],   # BMI = 26.3 → klasa 1
    [1.70, 60, 0],   # BMI = 20.8 → klasa 0
    [1.55, 70, 1],   # BMI = 29.1 → klasa 1
])

print(data[:,1] /data[:,0] ** 2)

# Podział na cechy (X) i etykiety (y)
X = data[:, :2]   # wzrost, waga
y = data[:, 2]    # klasa

# Dla testu możemy użyć np. wszystkich danych (albo podzielić na train/test)
X_test = X
y_test = y

print("X_test:\n", X_test)
print("y_test:", y_test)


In [ ]:
# 3. Ocena: dokładność na danych testowych
try:
    y_pred = perceptron.predict(X_test)
    acc = (y_pred == y_test).mean()
    print("Dokładność na teście:", round(float(acc), 3))
except NameError:
    print("Upewnij się, że zdefiniowałeś/aś i wytrenowałeś/aś obiekt 'perceptron'.")
